In [1]:
from __future__ import unicode_literals, print_function, division
import unicodedata
import string
import re
import random
import time
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import cPickle
import lstm_utils


import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

In [2]:
# Globals:
MAX_LENGTH = 25 # 25 token max
PREFIX_CONSTRAINT = False
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

PAD_BATCH_DATA = True
BATCH_SIZE = 16
learning_rate = 1e-3
num_epochs = 18
train_loss = []
test_loss = []


teacher_forcing_ratio = 0.5
use_gpu = torch.cuda.is_available()

In [3]:
SOS_token = 0
EOS_token = 1
PAD_token = 2
UNK_token = 3

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {"PAD":PAD_token, "SOS": SOS_token, "EOS": EOS_token, "UNK":UNK_token}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS", UNK_token: "UNK"}
        self.n_words = len(self.index2word)  # Next entry will be 4

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [5]:
def readLangs(lang1, reverse=False):
    print("Reading lines...")
    
    # Split every line by tabs into pairs and normalize
    pairs = cPickle.load(open('/Users/andrewsilva/Desktop/chat/my_word_embeds/pairs.pkl', 'rb'))

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        language = Lang(lang1)
    else:
        language = Lang(lang1)

    return language, pairs

In [6]:
def filterPair(p):
    # If we have a max length and we only want to consider sentences that start with X
    if MAX_LENGTH > 0 and PREFIX_CONSTRAINT:
        return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[1].startswith(eng_prefixes)
    # If we have a max length
    elif MAX_LENGTH > 0:
        return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH
    # If we only want to consider sentences that start with X
    elif PREFIX_CONSTRAINT:
        return p[1].startswith(eng_prefixes)
    else:
        return True

def filterPairs(pairs):
    # For each pair, only include ones that we want for this model
    return [pair for pair in pairs if filterPair(pair)]

In [7]:
def prepareData(lang1, reverse=False):
    # Create Lang instances, read in all data
    language, pairs = readLangs(lang1, reverse)
    for i in range(len(pairs)):
        pairs[i][0] = pairs[i][0].decode('utf-8').strip()
        pairs[i][1] = pairs[i][1].decode('utf-8').strip()
        
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    # Add pairs to each Lang instance (gives us word to index and index to word)
    for pair in pairs:
        language.addSentence(pair[0])
        language.addSentence(pair[1])
    print("Counted words:")
    print(language.name, language.n_words)
    return language, pairs


In [8]:
language, pairs = prepareData('english', True)
print(random.choice(pairs))
for pair in pairs:
    if len(pair[0].split()) > MAX_LENGTH:
        MAX_LENGTH = len(pair[0].split())
        print("PAIR0:", pair[0])
    if len(pair[1].split()) > MAX_LENGTH:
        MAX_LENGTH = len(pair[1].split())
        print("PAIR1:", pair[1])
print("Max length: %s" % MAX_LENGTH)

Reading lines...
Read 85988 sentence pairs
Trimmed to 52016 sentence pairs
Counted words:
english 9631
[u"haha she isn't a sticker person but she would definitely laugh", u'me too   \ufffci want this for your mom']
Max length: 25


In [9]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_gpu:
            return result.cuda()
        else:
            return result
    
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_gpu:
            return result.cuda()
        else:
            return result

In [10]:
# Return a list of indexes, one for each word in the sentence
def indexes_from_sentence(lang, sentence):
    indices = []
    if (type(sentence) is not unicode):
        sentence = unicode(sentence, "utf-8")
    sentence = normalizeString(sentence)
    for word in sentence.split():
        if word in lang.word2index:
            indices.append(lang.word2index[word])
        else:
            indices.append(lang.word2index['UNK'])
    return indices

def variable_from_sentence(lang, sentence):
    indexes = indexes_from_sentence(lang, sentence)
    indexes.append(EOS_token)
    var = Variable(torch.LongTensor(indexes).view(-1, 1))
#     print('var =', var)
    if use_gpu: var = var.cuda()
    return var

def variables_from_pair(pair):
    input_variable = variable_from_sentence(language, pair[0])
    target_variable = variable_from_sentence(language, pair[1])
    return (input_variable, target_variable)

In [36]:
teacher_forcing_ratio = 0.5
clip = 5.0

def train(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_variable.size()[0]
    target_length = target_variable.size()[0]

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_gpu else encoder_outputs

    loss = 0
    if input_length > MAX_LENGTH:
        print("too long")
        return 0
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_variable[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0][0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input.cuda() if use_gpu else decoder_input

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_variable[di])
            decoder_input = target_variable[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(1)
            ni = topi[0][0]

            decoder_input = Variable(torch.LongTensor([[ni]]))
            decoder_input = decoder_input.cuda() if use_gpu else decoder_input
            loss += criterion(decoder_output, target_variable[di])
            if ni == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.data[0] / target_length

In [37]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [38]:
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [39]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    input_variable = variable_from_sentence(language, sentence)
    input_length = input_variable.size()[0]
    encoder_hidden = encoder.initHidden()

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_gpu else encoder_outputs

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_variable[ei],
                                                 encoder_hidden)
        encoder_outputs[ei] = encoder_outputs[ei] + encoder_output[0][0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))  # SOS
    decoder_input = decoder_input.cuda() if use_gpu else decoder_input

    decoder_hidden = encoder_hidden

    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)

    for di in range(max_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        decoder_attentions[di] = decoder_attention.data
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if ni == EOS_token:
            decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(language.index2word[ni])

        decoder_input = Variable(torch.LongTensor([[ni]]))
        decoder_input = decoder_input.cuda() if use_gpu else decoder_input

    return decoded_words, decoder_attentions[:di + 1]

In [40]:
def evaluate_randomly(enc, dec):
    pair = random.choice(pairs)
    
    output_words, decoder_attn = evaluate(enc, dec, pair[0])
    output_sentence = ' '.join(output_words)
    
    print('>', pair[0])
    print('=', pair[1])
    print('<', output_sentence)
    print('')

In [41]:
# Configuring training
n_iters = 50000
plot_every = 200
print_every = 1000
learning_rate = 1e-3
hidden_size = 256
encoder = EncoderRNN(language.n_words, hidden_size)
decoder = AttnDecoderRNN(hidden_size, language.n_words, dropout_p=0.2)


if use_gpu:
    encoder = encoder.cuda()
    decoder = decoder.cuda()


In [ ]:
start = time.time()
plot_losses = []
print_loss_total = 0  # Reset every print_every
plot_loss_total = 0  # Reset every plot_every

encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
training_pairs = [variables_from_pair(random.choice(pairs))
                  for i in range(n_iters)]
criterion = nn.NLLLoss().cuda()

In [ ]:
for iteration in range(1, n_iters + 1):
    training_pair = training_pairs[iteration - 1]
    input_variable = training_pair[0]
    target_variable = training_pair[1]

    loss = train(input_variable, target_variable, encoder,
                 decoder, encoder_optimizer, decoder_optimizer, criterion)
    print_loss_total += loss
    plot_loss_total += loss

    if iteration % print_every == 0:
        print_loss_avg = print_loss_total / print_every
        print_loss_total = 0
        print('%s (%d %d%%) %.4f' % (timeSince(start, iteration / n_iters),
                                     iteration, iteration / n_iters * 100, print_loss_avg))

    if iteration % plot_every == 0:
        plot_loss_avg = plot_loss_total / plot_every
        plot_losses.append(plot_loss_avg)
        plot_loss_total = 0

showPlot(plot_losses)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:59: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


too long
too long
too long
too long
too long
too long
too long
too long
4m 29s (- 220m 25s) (1000 2%) 4.6575
too long
too long
too long
too long
too long
too long


In [ ]:
lstm_utils.save_checkpoint({
        'epoch':1,
        'encoder_state_dict': encoder.state_dict(),
        'decoder_state_dict': decoder.state_dict(),
        'encoder_optimizer': encoder_optimizer.state_dict(),
        'decoder_optimizer': decoder_optimizer.state_dict(),
    }, filename='me_chatbot.pth.tar')


In [ ]:
evaluate_randomly(encoder, decoder)

In [ ]:
output_words, attentions = evaluate(
    encoder, decoder, "what s up")
plt.matshow(attentions.numpy())

In [ ]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder, decoder, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("what s up")

evaluateAndShowAttention("how are you?")

evaluateAndShowAttention("what are you doing?")

evaluateAndShowAttention("where is kayla")